In [531]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [532]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

###Creating a learner
Things needed:
- linear block
- optimizer
- loss function
- accuracy metric

In [533]:
path1 = untar_data(URLs.MNIST)

In [534]:
Path.BASE_PATH = path1

In [535]:
(path1/'testing').ls().sorted()

(#10) [Path('testing/0'),Path('testing/1'),Path('testing/2'),Path('testing/3'),Path('testing/4'),Path('testing/5'),Path('testing/6'),Path('testing/7'),Path('testing/8'),Path('testing/9')]

In [536]:
path = untar_data(URLs.MNIST_SAMPLE)

In [537]:
def init_params(size, var=1):
    return torch.randn(size * var).requires_grad_()

In [538]:
def linear_model(xb, params):
    w, b = params
    return xb@w + b


In [539]:
def loss_func(preds, targs):
    return torch.where(targs==1, 1-preds, preds).mean()

In [540]:
def acc_metric(preds, targs):
    preds = preds.sigmoid()
    correct = (preds > 0.5) == targs
    return correct.float().mean()

In [541]:
# def calc_grad(xb, yb, params, model):
#     preds = model(xb, params)
#     loss = loss_func(preds, yb)
#     loss.backward()

In [542]:
Path.BASE_PATH = path

In [543]:
path.ls()

(#3) [Path('labels.csv'),Path('valid'),Path('train')]

In [544]:
threes = (path/'train'/'3').ls()
sevens = (path/'train'/'7').ls()
sevens

(#6265) [Path('train/7/35899.png'),Path('train/7/48909.png'),Path('train/7/18320.png'),Path('train/7/9510.png'),Path('train/7/22836.png'),Path('train/7/21159.png'),Path('train/7/6871.png'),Path('train/7/11919.png'),Path('train/7/55854.png'),Path('train/7/58042.png')...]

In [545]:
image_path = sevens[0]
image = Image.open(image_path)
image

In [546]:
list_threes = [tensor(Image.open(i)) for i in threes]
list_sevens = [tensor(Image.open(i)) for i in sevens]
len(list_threes), len(list_sevens)

(6131, 6265)

In [547]:
stacked_threes = torch.stack(list_threes).float()/255.0
stacked_sevens = torch.stack(list_sevens).float()/255.0
stacked_threes.shape, stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [548]:
x_train = torch.cat((stacked_threes, stacked_sevens)).view(-1,28*28)
y_train = tensor([[1]*len(stacked_threes) + [0]*len(stacked_sevens)]).view(-1,1)
x_train.shape, y_train.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [549]:
dset = list(zip(x_train, y_train)) 

In [550]:
valid_3 = (path/'valid'/'3').ls()
valid_7 = (path/'valid'/'7').ls()
valid_3 = torch.stack([tensor(Image.open(i)) for i in valid_3]).view(-1,28*28)/255.0
valid_7 = torch.stack([tensor(Image.open(i)) for i in valid_7]).view(-1,28*28)/255.0
valid_3_7 = torch.cat((valid_3,valid_7))
valid_label = tensor([1]*len(valid_3) + [0]*len(valid_7)).unsqueeze(1)

In [551]:
valid_set = list(zip(valid_3_7, valid_label))
valid_set = DataLoader(valid_set, bs=256)
xb, yb = first(valid_set)

In [552]:
### Creating data loaders from training set. 
dl = DataLoader(dset, bs=256)
xb,yb = first(dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [553]:
w = init_params((28*28,1))
b = init_params(1)
params = w,b
preds = linear_model(xb,params)

In [554]:
def train_epoch(model, lr, params):
    for x, y in dls:
        step_func(params)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [555]:
def validate_epoch(model):
    accs = [acc_metric(model(xb, params), yb) for xb,yb in valid_set]
    return round(torch.stack(accs).mean().item(), 4)

In [556]:
def calc_grad(xb, yb, model):
    preds = model(xb, params)
    loss = loss_func(preds, yb)
    loss.backward()

def train_epoch(model, lr, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [557]:
# for i in range(20):
#     train_epoch(linear_model, lr, params)
#     print(validate_epoch(linear_model), end=' ')

In [577]:
weights = init_params((28*28,1))
bias = init_params(1)
params = weights, bias

In [562]:
batch = x_train[:4]
batch.shape

# preds = linear_model(batch, params)
# preds
# (preds>0.0).float() == y_train[:4]

torch.Size([4, 784])

In [560]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()
batch_accuracy(linear_model(batch, params), y_train[:4])

tensor(0.2500)

In [561]:
validate_epoch(linear_model)

0.4174

In [578]:
xb = x_train[:300]
yb = y_train[:300]
lr = 0.001
params = weights,bias
train_epoch(linear_model, lr, params)
validate_epoch(linear_model)

0.6252

In [579]:
for i in range(40):
    train_epoch(linear_model, lr, params)
    print(validate_epoch(linear_model), end=" ")


0.6492 0.6718 0.6968 0.7232 0.7409 0.7591 0.7729 0.7822 0.7955 0.8023 0.8142 0.8283 0.8357 0.8445 0.8494 0.8543 0.8583 0.8612 0.8671 0.872 0.874 0.8799 0.8852 0.8906 0.8931 0.8961 0.898 0.898 0.901 0.9034 0.9069 0.9113 0.9143 0.9157 0.9167 0.9177 0.9187 0.9207 0.9231 0.9241 